In [1]:
import unittest
from blinker import signal, Signal
import silq
from silq.pulses import Pulse, PulseMatch

In [2]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run cell',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.restart_run_all();
        return false;
    }}
);

<IPython.core.display.Javascript object>

In [21]:
class TestPulseSignals(unittest.TestCase):
    def setUp(self):
        self.signal = signal('pulse')
        self.is_set = False 
        self.dict = {}
        
    def test_set_val(self):
        self.assertFalse(self.is_set)
        self.set_val(self)
        self.assertTrue(self.is_set)
        
    
    def test_signal_emit(self):
        p = Pulse()
        self.signal.connect(self.set_dict)
        self.assertFalse('t_start' in self.dict)
        p.t_start = 1
        self.assertEqual(1, self.dict['t_start'])
        
    def test_subsequent_pulses(self):
        p1 = Pulse(t_start=0, t_stop=10)
        self.assertEqual(p1.t_start, 0)
        self.assertEqual(p1.duration, 10)
        self.assertEqual(p1.t_stop, 10)
        
        p2 = Pulse(t_start=PulseMatch(p1, 't_stop', delay=1), duration=4)
        self.assertEqual(p2.t_start, 11)
        self.assertEqual(p2.duration, 4)
        self.assertEqual(p2.t_stop, 15)
        
        p1.t_stop = 14
        self.assertEqual(p1.t_start, 0)
        self.assertEqual(p1.duration, 14)
        self.assertEqual(p1.t_stop, 14)
        
        self.assertEqual(p2.t_start, 15)
        self.assertEqual(p2.duration, 4)
        self.assertEqual(p2.t_stop, 19)
        
        p2.t_start = 0
        self.assertEqual(p2.t_start, 0)
        self.assertEqual(p2.duration, 4)
        self.assertEqual(p2.t_stop, 4)
        
        p1.t_stop = 20
        self.assertEqual(p2.t_start, 0)
        self.assertEqual(p2.duration, 4)
        self.assertEqual(p2.t_stop, 4)
        
    def set_val(self, sender, **kwargs):
        self.is_set = True
        
    def set_dict(self, sender, **kwargs):
        self.dict.update(**kwargs)

In [22]:
# Run tests    
suite_pulse_signals = unittest.TestLoader().loadTestsFromModule(TestPulseSignals())
suite = unittest.TestSuite([suite_pulse_signals])
unittest.TextTestRunner(verbosity=2, failfast=True).run(suite)

test_set_val (__main__.TestPulseSignals) ... ok
test_signal_emit (__main__.TestPulseSignals) ... ok
test_subsequent_pulses (__main__.TestPulseSignals) ... 

disconnecting t_start: 15


ok

----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [18]:

p1 = Pulse(t_start=0, t_stop=14)
p2 = Pulse(t_start=PulseMatch(p1, 't_stop', delay=1), duration=4)

In [19]:
p2.t_start = 1
p1.t_stop = 20

disconnecting t_start: 1


In [20]:
p2.t_start

1

In [6]:
def outer_fun():
    def print_sender(sender):
        print(f'sender: {sender}')
    return print_sender

In [7]:
s = signal('pulse')

f1 = outer_fun()
f2 = outer_fun()
s.connect(f1)
s.connect(f2)

In [32]:
s.receivers

{}